In [ ]:
import warnings

warnings.filterwarnings("ignore", category=ImportWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="ipykernel")

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# %cd /content/drive/My Drive/AI Hack/data/
%cd /content/drive/MyDrive/AI\ Hack/data/

/content/drive/MyDrive/AI Hack/data


In [ ]:
!pip install pythermalcomfort

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 2.7 MB/s eta 0:00:00


In [ ]:
import os
from os.path import join
import numpy as np
import pandas as pd
import random
import time
from scipy.stats import linregress

random.seed(4738)

warnings.filterwarnings("ignore", category=ImportWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="ipykernel")

from pythermalcomfort.models import JOS3

In [ ]:
home = os.getcwd()
out = join(home, "output")
healthTable = pd.read_csv(join(home, "Health Data.csv"))
healthTable

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap

,Sex,Age,AvgHeight,RSEHeight,AvgWeight,RSEWeight,LowerHR1,UpperHR1,LowerHR2,UpperHR2,LowerHR3,UpperHR3,LowerHR4,UpperHR4
0,Male,18,176.6,0.2,81.1,1.1,82,100,70,73,61,65,56,61
1,Male,25,176.8,0.2,86.2,0.7,82,100,71,74,62,65,55,61
2,Male,35,175.7,0.2,88.5,0.8,83,100,71,75,63,66,57,62
3,Male,45,174.6,0.2,90.4,0.7,84,100,72,76,64,67,58,63
4,Male,55,173.2,0.2,89.4,0.7,82,100,72,75,62,67,57,61
5,Male,65,170.7,0.1,85.2,0.6,80,100,70,73,62,65,56,61
6,Female,18,163.2,0.2,65.9,1.4,85,100,74,78,66,69,61,65
7,Female,25,162.5,0.1,69.7,0.7,83,100,73,76,65,68,60,64
8,Female,35,162.4,0.1,73.2,0.8,85,100,74,78,65,69,60,64
9,Female,45,162.1,0.1,75.1,0.8,84,100,74,77,66,69,61,65


Assume we have 100 users, so we will have 100 personal health data. And if we got 10 time points of weather data (e.g. per-30-mins data for 5 hours), we will have 100 × 30 data points of perceived temperature.

In [ ]:
def getSexCode(sex):

  if sex == "Male":
    return 0

  elif sex == "Female":
    return 1

  else:
    return random.randint(0,1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def getStrokeVolume(sexCode, age):

  if sexCode == 0:
    coef1 = np.polyfit([30, 50, 70], [39, 37, 35], 0)
    # coef2 = np.polyfit([30, 50, 70], [91, 83, 75], 0)
    coef2 = np.polyfit([30, 50, 70], [104, 95, 87], 0)

  else:
    coef1 = np.polyfit([30, 50, 70], [34, 30, 27], 0)
    # coef2 = np.polyfit([30, 50, 70], [72, 66, 60], 0)
    coef2 = np.polyfit([30, 50, 70], [84, 78, 72], 0)

  low_lm = np.poly1d(coef1)
  up_lm = np.poly1d(coef2)

  lowSV = low_lm(age)/1000
  upSV = up_lm(age)/1000

  midSV = 0.3*lowSV + 0.7*upSV

  return np.random.choice([random.uniform(lowSV, midSV), random.uniform(midSV, upSV)], 1, p = [0.4, 0.6])[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def assumption_data_generator (N, sex):
  ### note: this is the draft framework of generating assumption data, could be amended.
  # N: number of data points
  # sex: the sex of the dataset
  sex = sex.title()
  sexCode = getSexCode(sex)
  chooseSex = True if sex == "Unknown" else False

  df = {"sex":[], "age":[], "height":[], "weight":[], "BMI":[], "freqOfExercise":[], "CI":[]}

  for i in range(N):

    # check if we should randomly pick a gender
    if chooseSex:
      sexCode = getSexCode(sex)
      sex = "Male" if sexCode == 0 else "Female"

    df["sex"].append(sexCode)

    # generate age depending on population
    if sex == 0:
      p_age = [0.09, 0.19, 0.17, 0.17, 0.16, 0.21]
      p_freq = [0.47, 0.23, 0.2, 0.1]
    else:
      p_age = [0.08, 0.19, 0.17, 0.17, 0.16, 0.23]
      p_freq = [0.66, 0.15, 0.12, 0.07]

    age = np.random.choice([random.randint(18,24), random.randint(25,34), random.randint(35,44), random.randint(45,54), random.randint(55,64), random.randint(65,86)], 1, p = p_age)[0]
    df["age"].append(age)

    # get the row id of the current data point
    rID = healthTable[(healthTable["Age"] <= age) & (healthTable["Sex"] == sex)].index[-1]

    # height
    height = round(np.random.normal(healthTable.loc[rID, "AvgHeight"], healthTable.loc[rID, "RSEHeight"]), 2)
    df["height"].append(height)

    # weight
    weight = round(np.random.normal(healthTable.loc[rID, "AvgWeight"], healthTable.loc[rID, "RSEWeight"]), 2)
    df["weight"].append(weight)

    # calculate the BMI
    df["BMI"].append(round(weight/((height/100)**2), 2))

    # generate the freq of exercise
    if age >= 80:
      freqExercise = random.randint(0,1)
    elif age >= 65:
      freqExercise = random.randint(0,2)
    else:
      freqExercise = np.random.choice([random.randint(0,1), random.randint(2,3), random.randint(4,5), random.randint(6,7)], 1, p = p_freq)[0]

    df["freqOfExercise"].append(freqExercise)

    ### calculation of Cardiac index
    # generate the Heart Rate
    freq_type = int(freqExercise/2) + 1
    lowHR = healthTable.loc[rID, "LowerHR"+str(freq_type)]
    upHR = healthTable.loc[rID, "UpperHR"+str(freq_type)]
    HR = random.randint(lowHR, upHR)

    # generate the Stroke Volume data
    SV = getStrokeVolume(sexCode, age)

    # calculate BSA
    BSA = np.sqrt(height*weight/3600)

    # calculate CI (normal range: 2.5~4.2 L/min/m^2)
    CI = (SV*HR) / BSA
    df["CI"].append(CI)

  return pd.DataFrame(df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data = pd.concat([assumption_data_generator(5, "Male"), assumption_data_generator(5, "Female")], ignore_index=True)
data.insert(0, "PersonID", range(1, len(data)+1))
data.to_csv(join(out, "example.csv"))
data.head()

,PersonID,sex,age,height,weight,BMI,freqOfExercise,CI
0,1,0,60,173.62,90.71,30.09,2,1.561767
1,2,0,74,170.66,84.89,29.15,2,3.339051
2,3,0,24,176.40,82.22,26.42,2,2.817813
3,4,0,24,176.75,81.71,26.16,0,4.362682
4,5,0,23,176.68,79.84,25.58,6,2.779703


TBD: how to generate data with unknown sex? One possible way is to randomly chose one of genders and gernate the data; the other is use the general distribution of height and weight.

In [ ]:
data

,PersonID,sex,age,height,weight,BMI,freqOfExercise,CI
0,1,0,60,173.62,90.71,30.09,2,1.561767
1,2,0,74,170.66,84.89,29.15,2,3.339051
2,3,0,24,176.40,82.22,26.42,2,2.817813
3,4,0,24,176.75,81.71,26.16,0,4.362682
4,5,0,23,176.68,79.84,25.58,6,2.779703
5,6,1,21,163.46,66.59,24.92,7,2.355144
6,7,1,70,157.52,71.68,28.89,2,2.335165
7,8,1,24,162.98,66.26,24.94,0,4.274825
8,9,1,58,160.61,74.25,28.78,1,3.382911
9,10,1,69,157.56,71.48,28.79,0,4.238961


In [ ]:
data.describe(include="all")

,PersonID,sex,age,height,weight,BMI,freqOfExercise,CI
count,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,5.50000,0.500000,44.700000,167.624000,76.963000,27.372000,2.200000,3.144802
std,3.02765,0.527046,23.137512,8.020524,8.149653,1.953907,2.440401,0.947340
min,1.00000,0.000000,21.000000,157.520000,66.260000,24.920000,0.000000,1.561767
25%,3.25000,0.000000,24.000000,161.202500,71.530000,25.725000,0.250000,2.461284
50%,5.50000,0.500000,41.000000,167.060000,77.045000,27.600000,2.000000,3.078432
75%,7.75000,1.000000,66.750000,175.705000,82.092500,28.865000,2.000000,4.024948
max,10.00000,1.000000,74.000000,176.750000,90.710000,30.090000,7.000000,4.362682


The following part is for combining actual weather data and assumption data.

In [ ]:
morningWeather = pd.read_csv(join(home, "MorningWeatherDataset.csv"), index_col=0)
morningWeather

,index,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,RainToday,RainTomorrow,WindDir9am,WindSpeed9am,Humidity9am,Pressure9am,Cloud9am,Temp9am
0,0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,0,ENE,6.0,20.0,1006.3,2.0,26.6
1,1,Cobar,27.1,36.1,0.0,13.0,0.0,N,43.0,No,0,N,7.0,26.0,1007.7,8.0,30.7
2,2,Cobar,23.3,34.0,0.0,9.8,12.6,SSW,41.0,No,0,S,17.0,33.0,1011.3,3.0,25.0
3,3,Cobar,16.1,34.2,0.0,14.6,13.2,SE,37.0,No,0,SE,15.0,25.0,1013.3,1.0,20.7
4,4,Cobar,19.0,35.5,0.0,12.0,12.3,ENE,48.0,No,0,ENE,30.0,46.0,1008.3,1.0,23.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4753,37631,Darwin,24.4,32.6,0.0,6.4,10.9,NNW,30.0,No,0,ENE,7.0,66.0,1012.4,1.0,29.1
4754,37632,Darwin,24.1,33.2,0.0,5.8,10.5,NW,30.0,No,0,SE,9.0,68.0,1012.8,1.0,29.5
4755,37633,Darwin,25.4,33.5,0.0,6.2,9.5,SSE,35.0,No,1,WSW,11.0,69.0,1010.8,1.0,29.9
4756,37634,Darwin,25.2,32.7,3.8,6.0,7.1,ENE,63.0,Yes,1,NE,9.0,79.0,1009.9,6.0,29.7


In [ ]:
afternoonWeather = pd.read_csv(join(home, "AfternoonWeatherDataset.csv"), index_col=0)
afternoonWeather

,index,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,RainToday,RainTomorrow,WindDir3pm,WindSpeed3pm,Humidity3pm,Pressure3pm,Cloud3pm,Temp3pm
0,0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,0,SW,20.0,13.0,1004.4,5.0,33.4
1,1,Cobar,27.1,36.1,0.0,13.0,0.0,N,43.0,No,0,WNW,20.0,19.0,1007.4,8.0,34.3
2,2,Cobar,23.3,34.0,0.0,9.8,12.6,SSW,41.0,No,0,SSE,19.0,15.0,1009.9,1.0,31.5
3,3,Cobar,16.1,34.2,0.0,14.6,13.2,SE,37.0,No,0,S,6.0,9.0,1009.2,1.0,32.8
4,4,Cobar,19.0,35.5,0.0,12.0,12.3,ENE,48.0,No,0,WSW,9.0,28.0,1004.0,5.0,33.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4753,37631,Darwin,24.4,32.6,0.0,6.4,10.9,NNW,30.0,No,0,NW,17.0,56.0,1009.0,1.0,32.2
4754,37632,Darwin,24.1,33.2,0.0,5.8,10.5,NW,30.0,No,0,WNW,22.0,57.0,1008.6,2.0,31.9
4755,37633,Darwin,25.4,33.5,0.0,6.2,9.5,SSE,35.0,No,1,NW,17.0,68.0,1007.6,3.0,28.8
4756,37634,Darwin,25.2,32.7,3.8,6.0,7.1,ENE,63.0,Yes,1,W,11.0,79.0,1006.6,6.0,27.6


In [ ]:
MorningData = pd.merge(morningWeather, data, how="cross")
#MorningData.to_csv(join(out, "MorningDataset.csv"))
MorningData

,index,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,RainToday,...,Cloud9am,Temp9am,PersonID,sex,age,height,weight,BMI,freqOfExercise,CI
0,0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,...,2.0,26.6,1,0,60,173.62,90.71,30.09,2,1.561767
1,0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,...,2.0,26.6,2,0,74,170.66,84.89,29.15,2,3.339051
2,0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,...,2.0,26.6,3,0,24,176.40,82.22,26.42,2,2.817813
3,0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,...,2.0,26.6,4,0,24,176.75,81.71,26.16,0,4.362682
4,0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,...,2.0,26.6,5,0,23,176.68,79.84,25.58,6,2.779703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47575,37635,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,...,7.0,28.1,6,1,21,163.46,66.59,24.92,7,2.355144
47576,37635,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,...,7.0,28.1,7,1,70,157.52,71.68,28.89,2,2.335165
47577,37635,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,...,7.0,28.1,8,1,24,162.98,66.26,24.94,0,4.274825
47578,37635,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,...,7.0,28.1,9,1,58,160.61,74.25,28.78,1,3.382911


In [ ]:
AfternoonData = pd.merge(afternoonWeather, data, how="cross")
#AfternoonData.to_csv(join(out, "AfternoonDataset.csv"))
AfternoonData

,index,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,RainToday,...,Cloud3pm,Temp3pm,PersonID,sex,age,height,weight,BMI,freqOfExercise,CI
0,0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,...,5.0,33.4,1,0,60,173.62,90.71,30.09,2,1.561767
1,0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,...,5.0,33.4,2,0,74,170.66,84.89,29.15,2,3.339051
2,0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,...,5.0,33.4,3,0,24,176.40,82.22,26.42,2,2.817813
3,0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,...,5.0,33.4,4,0,24,176.75,81.71,26.16,0,4.362682
4,0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,...,5.0,33.4,5,0,23,176.68,79.84,25.58,6,2.779703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47575,37635,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,...,6.0,30.3,6,1,21,163.46,66.59,24.92,7,2.355144
47576,37635,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,...,6.0,30.3,7,1,70,157.52,71.68,28.89,2,2.335165
47577,37635,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,...,6.0,30.3,8,1,24,162.98,66.26,24.94,0,4.274825
47578,37635,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,...,6.0,30.3,9,1,58,160.61,74.25,28.78,1,3.382911


Testing 1 input default "feels like" vs the model to see if there is any difference

In [ ]:
def personalizedFeelLike(sex, age, height, weight, bmi, ci, tempC, wind, humidity):
  # sex: all lower case
  # age: in years
  # height: in metres
  # weight: in kg
  # ci: cardiac index L/min/m^2
  # temp: in Celcius
  # wind: in metres per hour
  # humidity: in %

  # For men, the formula is (1.20 x BMI) + (0.23 x Age) – 16.2, while for women, it is (1.20 x BMI) + (0.23 x Age) – 5.4.
  # Not sure what to use if the sex is "unknown"
  if sex == 0:
    percentageFat = (1.20 * bmi) + (0.23 * age) - 16.2
  else:
    percentageFat = (1.20 * bmi) + (0.23 * age) - 5.4

  height = height / 100
  wind = wind * 1000


  model = JOS3(
      height=height,
      weight=weight,
      fat=percentageFat,
      age=age,
      sex=sex,
      ci=ci
  )

  model.tdb = tempC
  model.v = wind
  model.rh = humidity

  model.simulate(
      times=5,  # Number of loops of a simulation
      dtime=20,  # Time delta [sec]. The default is 60.
  )  # Exposure time = 30 [loops] * 60 [sec] = 30 [min]

  df = pd.DataFrame(model.dict_results())
  # df
  return df["t_skin_mean"].iloc[-1]


def defaultFeelsLike(tempC, wind, humidity):
  tempF = (tempC * 9/5) + 32
  windMiles = wind/1609

  feelsLikeF = 35.74 + (0.6215*tempF) - 35.75*(windMiles**0.16) + ((0.4275*tempF * windMiles**0.16))
  feelsLikeC = (feelsLikeF - 32) * 5/9
  return feelsLikeC

Add difference between **perceived temperature** and **temperature** into the dataset.

In [ ]:
perceivedTemps = []
for i in range(len(MorningData)):
  row = MorningData.iloc[i,]
  sex = row["sex"]
  age = row["age"]
  height = row["height"]
  weight = row["weight"]
  bmi = row["BMI"]
  ci = row["CI"]
  tempC = row["Temp9am"]
  wind = row["WindSpeed9am"]
  humidity = row["Humidity9am"]

  perTemp = personalizedFeelLike(sex, age, height, weight, bmi, ci, tempC, wind, humidity)

  perceivedTemps.append(perTemp)

MorningData["perceivedTemp"] = perceivedTemps
MorningData.to_csv(join(out, "MorningDataset.csv"))

In [ ]:
perceivedTemps = []
for i in range(len(AfternoonData)):
  row = AfternoonData.iloc[i,]
  sex = row["sex"]
  age = row["age"]
  height = row["height"]
  weight = row["weight"]
  bmi = row["BMI"]
  ci = row["CI"]
  tempC = row["Temp3pm"]
  wind = row["WindSpeed3pm"]
  humidity = row["Humidity3pm"]

  perTemp = personalizedFeelLike(sex, age, height, weight, bmi, ci, tempC, wind, humidity)

  perceivedTemps.append(perTemp)

AfternoonData["perceivedTemp"] = perceivedTemps
AfternoonData.to_csv(join(out, "AfternoonDataset.csv"))

KeyboardInterrupt: ignored

In [ ]:
AfternoonData["perceivedTemperature"] = AfternoonData.apply(lambda row : personalizedFeelLike(row["sex"], row["age"], row["height"], row["weight"], row["BMI"], row["CI"], row["Temp3pm"], row["WindSpeed3pm"], row["Humidity3pm"]), axis=1)
AfternoonData

In [ ]:
tempC = 20
wind = 20*1000 # in m/h
humidity = 64 # in %

defResults = defaultFeelsLike(tempC, wind, humidity)
print("Using default feels-like formula")
print(defResults)

sex = "male"
age = 86
height = 173.03
weight = 91.11
bmi = 30.43
ci = 2.4


josResults = personalizedFeelLike(sex, age, height, weight, bmi, ci, tempC, wind, humidity)
print("Using JOS-3")
print(josResults)


sex = "female"
age = 61
height = 160.69
weight = 74.6
bmi = 28.89
ci = 3.2

tempC = 20
wind = 20*1000 # in m/h
humidity = 64 # in %


josResults = personalizedFeelLike(sex, age, height, weight, bmi, ci, tempC, wind, humidity)
print("Using JOS-3")
print(josResults)

Using default feels-like formula
20.00248872116054
Using JOS-3
nan
Using JOS-3
19.68782642087596


In [ ]:
MorningData = pd.read_csv(join(out, "MorningDataset.csv"), index_col = 0).drop("index", axis = 1)
AfternoonData = pd.read_csv(join(out, "AfternoonDataset.csv"), index_col = 0).drop("index", axis = 1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
MorningData.columns = [col.replace("9am", "") for col in MorningData.columns]
MorningData

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,RainToday,RainTomorrow,...,Temp,PersonID,sex,age,height,weight,BMI,freqOfExercise,CI,perceivedTemp
0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,0,...,26.6,1,0,40,175.80,87.75,28.39,6,2.307144,24.794886
1,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,0,...,26.6,2,0,74,170.76,84.59,29.01,2,3.343987,24.793201
2,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,0,...,26.6,3,0,24,176.49,81.89,26.29,4,2.459838,24.797843
3,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,0,...,26.6,4,0,39,175.88,88.58,28.64,0,4.243744,24.792499
4,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,0,...,26.6,5,0,61,173.37,89.91,29.91,1,4.302597,24.789957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47575,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,1,...,28.1,6,1,37,162.47,74.20,28.11,5,2.384639,27.649656
47576,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,1,...,28.1,7,1,30,162.40,70.60,26.77,2,3.139030,27.648621
47577,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,1,...,28.1,8,1,34,162.24,69.47,26.39,1,2.461945,27.650609
47578,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,1,...,28.1,9,1,22,163.19,66.04,24.80,0,4.014209,27.649452


In [ ]:
MorningData.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustDir', 'WindGustSpeed', 'RainToday', 'RainTomorrow', 'WindDir',
       'WindSpeed', 'Humidity', 'Pressure', 'Cloud', 'Temp', 'PersonID', 'sex',
       'age', 'height', 'weight', 'BMI', 'freqOfExercise', 'CI',
       'perceivedTemp'],
      dtype='object')

In [ ]:
AfternoonData.columns = [col.replace("3pm", "") for col in AfternoonData.columns]
AfternoonData

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,RainToday,RainTomorrow,...,Temp,PersonID,sex,age,height,weight,BMI,freqOfExercise,CI,perceivedTemp
0,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,0,...,33.4,1,0,40,175.80,87.75,28.39,6,2.307144,30.022706
1,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,0,...,33.4,2,0,74,170.76,84.59,29.01,2,3.343987,30.022508
2,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,0,...,33.4,3,0,24,176.49,81.89,26.29,4,2.459838,30.024152
3,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,0,...,33.4,4,0,39,175.88,88.58,28.64,0,4.243744,30.019678
4,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,No,0,...,33.4,5,0,61,173.37,89.91,29.91,1,4.302597,30.019773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47575,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,1,...,30.3,6,1,37,162.47,74.20,28.11,5,2.384639,29.336127
47576,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,1,...,30.3,7,1,30,162.40,70.60,26.77,2,3.139030,29.335465
47577,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,1,...,30.3,8,1,34,162.24,69.47,26.39,1,2.461945,29.337109
47578,Darwin,22.9,33.4,17.8,7.0,6.2,E,93.0,Yes,1,...,30.3,9,1,22,163.19,66.04,24.80,0,4.014209,29.336220


In [ ]:
AfternoonData.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustDir', 'WindGustSpeed', 'RainToday', 'RainTomorrow', 'WindDir',
       'WindSpeed', 'Humidity', 'Pressure', 'Cloud', 'Temp', 'PersonID', 'sex',
       'age', 'height', 'weight', 'BMI', 'freqOfExercise', 'CI',
       'perceivedTemp'],
      dtype='object')

In [ ]:
WeatherData = pd.concat([MorningData, AfternoonData])
WeatherData["Target"] = WeatherData["perceivedTemp"] - WeatherData["Temp"]
WeatherData = WeatherData.drop(["Location", "WindGustDir", "WindDir", "perceivedTemp"], axis = 1)
WeatherData["RainToday"] = [0 if i == "No" else 1 for i in WeatherData["RainToday"]]
WeatherData.to_csv(join(out, "WeatherData.csv"))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [1]:
import requests

url = "https://weatherapi-com.p.rapidapi.com/current.json"

querystring = {"q":"53.1,-0.13"}

headers = {
	"X-RapidAPI-Key": "beacf3c7cfmsh084cbf3d0bd8766p1c9c54jsne1fc399a50b8",
	"X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'location': {'name': 'Boston', 'region': 'Lincolnshire', 'country': 'United Kingdom', 'lat': 53.1, 'lon': -0.13, 'tz_id': 'Europe/London', 'localtime_epoch': 1700792619, 'localtime': '2023-11-24 2:23'}, 'current': {'last_updated_epoch': 1700792100, 'last_updated': '2023-11-24 02:15', 'temp_c': 8.0, 'temp_f': 46.4, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 18.6, 'wind_kph': 29.9, 'wind_degree': 300, 'wind_dir': 'WNW', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 71, 'cloud': 0, 'feelslike_c': 4.3, 'feelslike_f': 39.8, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 26.3, 'gust_kph': 42.4}}


In [3]:
import pandas as pd
pd.DataFrame(response.json())

,location,current
name,Boston,NaN
region,Lincolnshire,NaN
country,United Kingdom,NaN
lat,53.1,NaN
lon,-0.13,NaN
tz_id,Europe/London,NaN
localtime_epoch,1700792619,NaN
localtime,2023-11-24 2:23,NaN
last_updated_epoch,NaN,1700792100
last_updated,NaN,2023-11-24 02:15


temp_c, wind_kph, pressure_mb, precip_mm,